In [29]:
import os
import websockets
import json
import pandas as pd
import asyncio
import ta

In [59]:
# Initialise env variables
API_KEY = os.environ.get('BINANCE_API_KEY')
API_SECRET = os.environ.get('BINANCE_API_SECRET')

# Init buy / sell vars
CRYPTO = 'BTCUSDT'
CRYPTO_PRICE = float(client.get_symbol_ticker(symbol="BTCUSDT")['price'])
BUY_QTY_USD = 10
BUY_QTY_CRYPTO = round(BUY_QTY_USD / CRYPTO_PRICE, 8)
print(f'Amount of {CRYPTO} to buy: {BUY_QTY_CRYPTO}')

Amount of BTCUSDT to buy: 0.00015723


In [5]:
stream = websockets.connect('wss://stream.binance.com:9443/stream?streams=adausdt@miniTicker')

In [40]:
def pretty(obj):
    print(json.dumps(obj, indent=2))

In [32]:
def createFrame(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','c']]
    df.columns = ['symbol', 'Time', 'Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [ ]:
df = pd.DataFrame()
open_position = False

async with stream as receiver:
    while True:
        data = await receiver.recv()
        data = json.loads(data)['data']
        df = df.append(createFrame(data))
        
        if len(df) > 30:
            if not open_position:
                if ta.momentum.roc(df.Price, 30).iloc[-1] > 0 and ta.momentum.roc(df.Price, 30).iloc[-2]:
                    order= client.create_order(symbol = 'ADAUSDT', side = 'BUY', type = 'MARKET', quantity = 50)
                    print(order)
                    open_position = True
                    buyPrice = float(order['fills'][0]['price'])
            if open_position:
                subdf = df[df.Time >= pd.to_datetime(order['transactTime'], unit='ms')]
                if len(subdf) > 1:
                    subdf['highest'] = subdf.Price.cummax()
                    subdf['trailingstop'] = subdf['highest'] * 0.995
                    if subdf.iloc[-1].Price < subdf.iloc[-1].trailingstop or df.iloc[-1].Price / float(order['fills'][0]['price'] > 1.002:
                        order = client.create_order(symbol = 'ADAUSDT', side = 'SELL', type = 'MARKET', quantity = 50)
                        print(order)
                        sellPrice = float(order['fills'][0]['price'])
                        print(f"You made {(sellPrice - buyPrice)/buyPrice} profit.")
                        open_position = False
        print(df.iloc[-1])

In [31]:
from binance.client import Client

client = Client(API_KEY, API_SECRET)

In [ ]:
pretty(client.get_symbol_info('BTCUSDT'))
pretty(float(client.get_symbol_ticker(symbol="BTCUSDT")['price']))
# pretty(info)

In [ ]:
8, 6.5, 5, 3.5, 2, 0.5, 11.